In [250]:
text=input()
# text='一家人在吃饭。 儿子问：“爸爸，虫子能吃吗？” 爸爸说：“儿子， 妈妈做的饭菜好吃吗？” 儿子说：“很好吃！” 爸爸说：“那么，你好好吃饭。 吃饭的时候，不要说话。好吗？” 儿子说：“好的。”'
# text='在一个商店,他看到一只小猫,对不起,大家到了清华大学'
text

 2.  英男：这双鞋是多大号的？  售货员：25号的。  英男：我可以试试吗？  售货员：可以。  英男：这双有点儿小，有大一点儿的吗？  售货员：有，您再试试25号半的。  英男：这双不大也不小，挺合适。    3.  卖东西的：您想买点儿什么？  莉莉：我看看那件白的真丝衬衣。有别的颜色的吗？  卖东西的：没有，只有这一种颜色。  莉莉：多少钱一件？  卖东西的：一百八。  莉莉：太贵了，便宜点儿吧。  卖东西的：您给一百六吧。  莉莉：再便宜点儿，一百五怎么样？  卖东西的：行。


'2.  英男：这双鞋是多大号的？  售货员：25号的。  英男：我可以试试吗？  售货员：可以。  英男：这双有点儿小，有大一点儿的吗？  售货员：有，您再试试25号半的。  英男：这双不大也不小，挺合适。    3.  卖东西的：您想买点儿什么？  莉莉：我看看那件白的真丝衬衣。有别的颜色的吗？  卖东西的：没有，只有这一种颜色。  莉莉：多少钱一件？  卖东西的：一百八。  莉莉：太贵了，便宜点儿吧。  卖东西的：您给一百六吧。  莉莉：再便宜点儿，一百五怎么样？  卖东西的：行。'

In [251]:
import json
import termcolor
import jieba

with open('hsk-level-1.json') as file:
    hsk1_data = json.load(file)
hsk1_words = []
for item in hsk1_data:
    hsk1_words.append(item['hanzi'])

with open('hsk-level-2.json') as file:
    hsk2_data = json.load(file)
hsk2_words = []
for item in hsk2_data:
    hsk2_words.append(item['hanzi'])

with open('hsk-level-3.json') as file:
    hsk3_data = json.load(file)
hsk3_words = []
for item in hsk3_data:
    hsk3_words.append(item['hanzi'])

In [259]:
tagged_words_hsk1=[]
tagged_words_hsk2=[]
tagged_words_hsk3=[]

print("using jieba for word segmentation:")
for word in jieba.cut(text, cut_all=False):
    #cut word in small pieces
    print(word,len(word))
    # for each word output from jieba, check the subset of it
    subset_of_word=[]
    if len(word) >= 4:
#         print(word,4)
        subset_of_word.append(word[0])
        subset_of_word.append(word[1])
        subset_of_word.append(word[2])
        subset_of_word.append(word[3])
        subset_of_word.append(word[0:2])
        subset_of_word.append(word[1:3])
        subset_of_word.append(word[2:4])
        subset_of_word.append(word[0:3])
        subset_of_word.append(word[1:4])
    elif len(word) >= 3:
#         print(word,3)
        subset_of_word.append(word[0])
        subset_of_word.append(word[1])
        subset_of_word.append(word[2])
        subset_of_word.append(word[0:2])
        subset_of_word.append(word[1:3])
    elif len(word)>=2:
#         print(word,2)
        subset_of_word.append(word[0])
        subset_of_word.append(word[1])

# check the word directly from jieba    
    if word in hsk1_words and word not in tagged_words_hsk1:
        tagged_words_hsk1.append(word)
    elif word in hsk2_words and word not in tagged_words_hsk2:
        tagged_words_hsk2.append(word)
    elif word in hsk3_words and word not in tagged_words_hsk3:
        tagged_words_hsk3.append(word)
    
    
# also check subset of the word 
    for i in subset_of_word:
#         print(i)
        if i in hsk1_words and i not in tagged_words_hsk1:
            tagged_words_hsk1.append(i)
        if i in hsk2_words and i not in tagged_words_hsk2:
            tagged_words_hsk2.append(i)
        if i in hsk3_words and i not in tagged_words_hsk3:
            tagged_words_hsk3.append(i)
print("=======================")
print("HSK1:",tagged_words_hsk1)
print("HSK2:",tagged_words_hsk2)
print("HSK3:",tagged_words_hsk3)

using jieba for word segmentation:
2 1
. 1
  1
  1
英男 2
： 1
这双鞋 3
是 1
多大号 3
的 1
？ 1
  1
  1
售货员 3
： 1
25 2
号 1
的 1
。 1
  1
  1
英男 2
： 1
我 1
可以 2
试试 2
吗 1
？ 1
  1
  1
售货员 3
： 1
可以 2
。 1
  1
  1
英男 2
： 1
这 1
双 1
有点儿 3
小 1
， 1
有大 2
一点儿 3
的 1
吗 1
？ 1
  1
  1
售货员 3
： 1
有 1
， 1
您 1
再 1
试试 2
25 2
号 1
半 1
的 1
。 1
  1
  1
英男 2
： 1
这双 2
不大 2
也 1
不小 2
， 1
挺 1
合适 2
。 1
  1
  1
  1
  1
3 1
. 1
  1
  1
卖东西 3
的 1
： 1
您 1
想 1
买点儿 3
什么 2
？ 1
  1
  1
莉莉 2
： 1
我 1
看看 2
那件 2
白 1
的 1
真丝 2
衬衣 2
。 1
有 1
别的 2
颜色 2
的 1
吗 1
？ 1
  1
  1
卖东西 3
的 1
： 1
没有 2
， 1
只有 2
这 1
一种 2
颜色 2
。 1
  1
  1
莉莉 2
： 1
多少 2
钱 1
一件 2
？ 1
  1
  1
卖东西 3
的 1
： 1
一百八 3
。 1
  1
  1
莉莉 2
： 1
太贵 2
了 1
， 1
便宜 2
点儿 2
吧 1
。 1
  1
  1
卖东西 3
的 1
： 1
您 1
给 1
一百 2
六 1
吧 1
。 1
  1
  1
莉莉 2
： 1
再 1
便宜 2
点儿 2
， 1
一百 2
五 1
怎么样 3
？ 1
  1
  1
卖东西 3
的 1
： 1
行 1
。 1
HSK1: ['这', '是', '多', '大', '的', '我', '吗', '有', '点', '小', '一', '不', '东西', '想', '买', '什么', '看', '那', '没', '多少', '少', '钱', '八', '太', '了', '六', '五', '怎么样', '怎么']
HSK2: ['号', '可以', '您', '再', '也', '

In [253]:
# Create list of flags for each HSK level

# initialize flag as list of 0
hsk1_flag=[0]*len(text)
hsk2_flag=[0]*len(text)
hsk3_flag=[0]*len(text)


def tag(flag_list_name,starting_position, length, hsk_level):
    ## flag a slice of list according to the length
    for i in range(length):
        flag_list_name[starting_position+i]=hsk_level
        None



for cursor_position in enumerate(text):
# test word from one syllable to 4 syllables, flag of longer word will override short word in the same level

    window=text[cursor_position[0]:cursor_position[0]+4]
    if len(window) != 4:
        None
    elif window in tagged_words_hsk1:
        tag(hsk1_flag,cursor_position[0],4,1)
    elif window in tagged_words_hsk2:
        tag(hsk2_flag,cursor_position[0],4,2)
    elif window in tagged_words_hsk3:
        tag(hsk3_flag,cursor_position[0],4,3)
        
    window=text[cursor_position[0]:cursor_position[0]+3]        
    if len(window) != 3:
        None    
    elif window in tagged_words_hsk1:
        print(window) 
        tag(hsk1_flag,cursor_position[0],3,1)
    elif window in tagged_words_hsk2:
        tag(hsk2_flag,cursor_position[0],3,2)
    elif window in tagged_words_hsk3:
        tag(hsk3_flag,cursor_position[0],3,3)

    window=text[cursor_position[0]:cursor_position[0]+2]
    if len(window) != 2:
        None
    elif window in tagged_words_hsk1:
        print(window) 
        tag(hsk1_flag,cursor_position[0],2,1)
    elif window in tagged_words_hsk2:
        tag(hsk2_flag,cursor_position[0],2,2)
    elif window in tagged_words_hsk3:
        tag(hsk3_flag,cursor_position[0],2,3)

    window=text[cursor_position[0]:cursor_position[0]+1]    
    if window in tagged_words_hsk1:
        tag(hsk1_flag,cursor_position[0],1,1)
    elif window in tagged_words_hsk2:
        tag(hsk2_flag,cursor_position[0],1,2)
    elif window in tagged_words_hsk3:
        tag(hsk3_flag,cursor_position[0],1,3)



# check tagging result for each HSK level

for i in enumerate(text):
    print(i[0],text[i[0]],hsk1_flag[i[0]], hsk2_flag[i[0]], hsk3_flag[i[0]])

东西
什么
东西
多少
东西
东西
怎么样
怎么
东西
0 2 0 0 0
1 . 0 0 0
2   0 0 0
3   0 0 0
4 英 0 0 0
5 男 0 0 0
6 ： 0 0 0
7 这 1 0 0
8 双 0 0 3
9 鞋 0 0 3
10 是 1 0 0
11 多 1 0 0
12 大 1 0 0
13 号 0 2 0
14 的 1 0 0
15 ？ 0 0 0
16   0 0 0
17   0 0 0
18 售 0 0 0
19 货 0 0 0
20 员 0 0 0
21 ： 0 0 0
22 2 0 0 0
23 5 0 0 0
24 号 0 2 0
25 的 1 0 0
26 。 0 0 0
27   0 0 0
28   0 0 0
29 英 0 0 0
30 男 0 0 0
31 ： 0 0 0
32 我 1 0 0
33 可 0 2 0
34 以 0 2 0
35 试 0 0 0
36 试 0 0 0
37 吗 1 0 0
38 ？ 0 0 0
39   0 0 0
40   0 0 0
41 售 0 0 0
42 货 0 0 0
43 员 0 0 0
44 ： 0 0 0
45 可 0 2 0
46 以 0 2 0
47 。 0 0 0
48   0 0 0
49   0 0 0
50 英 0 0 0
51 男 0 0 0
52 ： 0 0 0
53 这 1 0 0
54 双 0 0 3
55 有 1 0 0
56 点 1 0 0
57 儿 0 0 0
58 小 1 0 0
59 ， 0 0 0
60 有 1 0 0
61 大 1 0 0
62 一 1 0 0
63 点 1 0 0
64 儿 0 0 0
65 的 1 0 0
66 吗 1 0 0
67 ？ 0 0 0
68   0 0 0
69   0 0 0
70 售 0 0 0
71 货 0 0 0
72 员 0 0 0
73 ： 0 0 0
74 有 1 0 0
75 ， 0 0 0
76 您 0 2 0
77 再 0 2 0
78 试 0 0 0
79 试 0 0 0
80 2 0 0 0
81 5 0 0 0
82 号 0 2 0
83 半 0 0 3
84 的 1 0 0
85 。 0 0 0
86   0 0 0
87   0 0 0
88 英 0 0 0
89 

In [254]:
## combine flags and assign font color and background color to each character
# Available text colors: red, green, yellow, blue, magenta, cyan, white.
HSK1_color = 'red'
HSK2_color = 'green'
HSK3_color = 'yellow'

combined_flag = []
for i in enumerate(text):
    d = {'character':text[i[0]],'font_color':None, 'bg_color':None}
    combined_flag.append(d)

for (cursor_position,character) in enumerate(text):
#     print(cursor_position, character,hsk1_flag[cursor_position])
    if hsk1_flag[cursor_position] != 0:
        combined_flag[cursor_position]['font_color'] = HSK1_color
    # for higher HSK level word, first check if it is already tagged. If so, using background color.
    if hsk2_flag[cursor_position] != 0:
        if combined_flag[cursor_position]['font_color'] == None:
            combined_flag[cursor_position]['font_color'] = HSK2_color
        elif combined_flag[cursor_position]['bg_color'] == None:
            combined_flag[cursor_position]['bg_color'] = 'on_' + HSK2_color

    if hsk3_flag[cursor_position] != 0:
        if combined_flag[cursor_position]['font_color'] == None:
            combined_flag[cursor_position]['font_color'] = HSK3_color
        elif combined_flag[cursor_position]['bg_color'] == None:
            combined_flag[cursor_position]['bg_color'] = 'on_' + HSK3_color
    


In [255]:

### old code below
# while position<len(text):
#     print(hsk2_flag[position])
#     if hsk2_flag[position] == 4:
#         combined_flag[position]['font_color']=HSK2_color
#     elif hsk2_flag[position] == 3:
#         combined_flag[position]['font_color']=HSK2_color
#     elif hsk2_flag[position] == 2:
#         combined_flag[position]['font_color']=HSK2_color
#     elif hsk2_flag[position] == 1:
#         combined_flag[position]['font_color']=HSK2_color

#     if hsk1_flag[position] == 4:
#         combined_flag[position]['font_color']=HSK1_color
#         combined_flag[position+1]['font_color']=HSK1_color
#         combined_flag[position+2]['font_color']=HSK1_color
#         combined_flag[position+3]['font_color']=HSK1_color
#         position +=4
#     elif hsk1_flag[position] == 3:
#         combined_flag[position]['font_color']=HSK1_color
#         combined_flag[position+1]['font_color']=HSK1_color
#         combined_flag[position+2]['font_color']=HSK1_color
#         position +=3
#     elif hsk1_flag[position] == 2:
#         combined_flag[position]['font_color']=HSK1_color
#         combined_flag[position+1]['font_color']=HSK1_color
#         position +=2
#     elif hsk1_flag[position] == 1:
#         combined_flag[position]['font_color']=HSK1_color
#         position +=1
#     else: 
#         position +=1

    

In [256]:
# check combined_flag
# combined_flag

In [261]:
# output text according to the combined flag
print("Colored text (red for HSK1, green for HSK2, yellow for HSK3):\n---")
for i in enumerate(text):
#     print(i,combined_flag[i[0]]['font_color'], combined_flag[i[0]]['bg_color'])
    colored_word = termcolor.colored(i[1], color=combined_flag[i[0]]['font_color'], on_color=combined_flag[i[0]]['bg_color'])
    print(colored_word, end="")
print("\n---")
print('HSK3 words:', tagged_words_hsk3)
print('HSK2 words:', tagged_words_hsk2)
print('HSK1 words:', tagged_words_hsk1)

Colored text (red for HSK1, green for HSK2, yellow for HSK3):
---
2.  英男：这双鞋是多大号的？  售货员：25号的。  英男：我可以试试吗？  售货员：可以。  英男：这双有点儿小，有大一点儿的吗？  售货员：有，您再试试25号半的。  英男：这双不大也不小，挺合适。    3.  卖东西的：您想买点儿什么？  莉莉：我看看那件白的真丝衬衣。有别的颜色的吗？  卖东西的：没有，只有这一种颜色。  莉莉：多少钱一件？  卖东西的：一百八。  莉莉：太贵了，便宜点儿吧。  卖东西的：您给一百六吧。  莉莉：再便宜点儿，一百五怎么样？  卖东西的：行。
---
HSK3 words: ['双', '鞋', '半', '东', '西', '只', '种']
HSK2 words: ['号', '可以', '您', '再', '也', '卖', '件', '白', '真', '别', '颜色', '百', '贵', '便宜', '吧', '给']
HSK1 words: ['这', '是', '多', '大', '的', '我', '吗', '有', '点', '小', '一', '不', '东西', '想', '买', '什么', '看', '那', '没', '多少', '少', '钱', '八', '太', '了', '六', '五', '怎么样', '怎么']
